# Homework 5

This homework asks you to perform various experiments with ensemble methods. 

The dataset is the same real estate dataset we previously used from:

https://www.kaggle.com/datasets/mirbektoktogaraev/madrid-real-estate-market

You will write code and discussion into code and text cells in this notebook. 

If a code block starts with TODO:, this means that you need to write something there. 

There are also markdown blocks with questions. Write the answers to these questions in the specified locations.

Some code had been written for you to guide the project. Don't change the already written code.

## Grading
The points add up to 10. Extensive partial credit will be offered. Thus, make sure that you are at least attempting all problems. 

Make sure to comment your code, such that the grader can understand what different components are doing or attempting to do. 

In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble


# A. Setup. 

In this project we are going to work in a multi-variable setting. 

This time, there are 7 explanatory variables: ``sq_mt_built``, ``n_rooms``, ``n_bathrooms``, ``is_renewal_needed``, ``is_new_development`` and ``has_fitted_wardrobes``. 

We will first create the training and test data while doing some minimal data cleaning.

In [2]:
df = pd.read_csv("houses_Madrid.csv")
#print(f"The columns of the database {df.columns}")

xfields = ["sq_mt_built", "n_rooms", "n_bathrooms", "has_individual_heating", \
           "is_renewal_needed", "is_new_development", "has_fitted_wardrobes"]
yfield = ["buy_price"]
# print (xfields + yfield)
dfsel = df[xfields + yfield]
dfselnona = dfsel.dropna()
df_shuffled = dfselnona.sample(frac=1) # shuffle the rows
x = df_shuffled[xfields].to_numpy(dtype=np.float64)
y = df_shuffled[yfield].to_numpy(dtype=np.float64)
print(x.shape)
training_data_x = x[:8000]
training_data_y = y[:8000]
test_data_x = x[8000:]
test_data_y = y[8000:]
print(f"Training data is composed of {len(training_data_x)} samples.")
print(f"Test data is composed of {len(test_data_x)} samples.")
# print(test_data_x[45])

(9764, 7)
Training data is composed of 8000 samples.
Test data is composed of 1764 samples.


# B. Creating a linear regression multi-variable baseline. 

In this section we make a linear regression predictor for the multi-variable case. We also check the performance of the resulting regressor, and print the error. 

This part is had been done for you, such that the work does not depend on you importing parts from the previous projects. 

You will need to adapt this for the other models. 

In [3]:
# training the linear regressor
regressor = sklearn.linear_model.LinearRegression()
regressor.fit(training_data_x, training_data_y)
# We will create the predictions yhat for every x from the training data. We will do this one at a time. This is not an efficient way to do it, but it allows you to write and debug functions that return a scalar number
yhats = []
for x in test_data_x:
    yhat = regressor.predict([x])
    yhats.append(yhat[0])

# Now, print some examples of the quality of the classifier
examples = [45, 67, 170, 189, 207]
for i in examples:
    x = test_data_x[i]
    y = test_data_y[i]
    yhat = regressor.predict([x])[0][0]
    print(f"House {i} with {x[0]} sqmt was sold for {y} euros, but our system predicted {yhat:.2f}")

# Now calculate the root mean square error on the resulting arrays
error = sklearn.metrics.mean_squared_error(yhats, test_data_y, squared=False)
print(f"The mean square error of the linear regression is {error:.2f} euro")

House 45 with 142.0 sqmt was sold for [650000.] euros, but our system predicted 485771.25
House 67 with 85.0 sqmt was sold for [340000.] euros, but our system predicted 399139.28
House 170 with 48.0 sqmt was sold for [255000.] euros, but our system predicted 429230.64
House 189 with 96.0 sqmt was sold for [170000.] euros, but our system predicted 459119.81
House 207 with 178.0 sqmt was sold for [530000.] euros, but our system predicted 743055.71
The mean square error of the linear regression is 388484.49 euro


# P1: Random Forest using sklearn (5 points)

Use the RandomForestRegressor function from sklearn to predict the prices of the house. Print the resulting error and samples, similar to the way in Section B. 

Experiment with the settings of the hyperparameters: n_estimators (try at least values 10, 25, 100, 200) and max_depth (try at least values 1, 2, 4, 8, 16 and None).

Retain the hyperparameter value that gives you the best result. 



In [9]:
# TODO implement here
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#Define the hyperparameter
n_estimator_values = [10,25,100,200]
max_depth_values = [1,2,4,8,16, None]

best_score = float('inf')
best_n_estimators = None
best_max_depth = None


#Loop through different hyperparameter combinations
for n_estimators in n_estimator_values:
    for max_depth in max_depth_values:
        #Create and train the model
        model = RandomForestRegressor(n_estimators= n_estimators, max_depth=max_depth, random_state=42)
        model.fit(training_data_x,training_data_y)

        #Making the predictions
        y_pred = model.predict(test_data_x)

        #Calculate the mse error
        mse = mean_squared_error(test_data_y, y_pred, squared=False)

         # Print error and sample predictions
        #print(f"n_estimators={n_estimators}, max_depth={max_depth}, Mean Squared Error: {mse}")
        
        # Check if this model is better than the current best
        if mse < best_score:
            best_score = mse
            best_n_estimators = n_estimators
            best_max_depth = max_depth

print(f"\nBest Hyperparameters: n_estimators={best_n_estimators}, max_depth={best_max_depth}, Mean Squared Error: {best_score:.2f}")


/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was exp


Best Hyperparameters: n_estimators=200, max_depth=8, Mean Squared Error: 367109.43


# Questions: 
* Q: Do you find that Random Forest performs better than the previous approaches you implemented? Discuss. 
* A: Yes it does because the MSE compared to the previous approaches, the MSE is smaller compared to the previous approach.

* Q: Explain the impact of the number of estimators and max tree depth hyperparameters on the accuracy. Which hyperparameter setting gives you the best value? Is this the same as the default settings in sklearn?

* A:The number of estimators (trees) and the max tree depth are crucial hyperparameters in Random Forest that significantly impact accuracy. Generally, increasing the number of estimators can improve model performance up to a certain point, after which the improvement may plateau or even decrease due to overfitting.

* Q: Explain the impact of the hyperparameters on the training time. 
* A: The training time in Random Forest is influenced by various factors, including the number of estimators, the size of the dataset, and the complexity of the trees (controlled by the max tree depth and other hyperparameters). Generally, increasing the number of estimators and the max tree depth tends to increase training time because it requires fitting more trees and building deeper trees. However, Random Forest training is inherently parallelizable, allowing for efficient training on multicore processors. Therefore, while increasing these hyperparameters may increase training time, it might not scale linearly with the increase in hyperparameter values.


# P2: AdaBoost using sklearn (5 points)

Use the AdaBoost function from sklearn to predict the prices of the house. Print the resulting error and samples, similar to the way in Section B. 

Experiment with the settings of the hyperparameters: loss (try "linear", "square" and "exponential) and learning_rate (try at least values 0.2, 0.5, 1 and 2)

In [10]:
# TODO implement here
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

# Define hyperparameter values to experiment with
loss_values = ["linear", "square", "exponential"]
learning_rate_values = [0.2, 0.5, 1, 2]

best_score = float('inf')
best_loss = None
best_learning_rate = None

#Loop through the different hyperparameter values
for loss in loss_values:
    for learning_rate in learning_rate_values:
        #Create and train the model
        model = AdaBoostRegressor(loss=loss, learning_rate=learning_rate, random_state=42)
        model.fit(training_data_x,training_data_y)

        #Mean_sqaured_error
        mse = mean_squared_error(test_data_y, y_pred= model.predict(test_data_x), squared=False)
        
        # Check if this model is better than the current best
        if mse < best_score:
            best_score = mse
            best_loss = loss
            best_learning_rate = learning_rate

print(f"\nBest Hyperparameters: loss={best_loss}, learning_rate={best_learning_rate}, Mean Squared Error: {best_score:.2f}")


/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/albertjojo/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array w


Best Hyperparameters: loss=square, learning_rate=1, Mean Squared Error: 405897.85


# Questions: 
* Q: Do you find that Adaboost performs better than the previous approaches you implemented? Discuss. 

* A: No, Adaboost does not perform better than the previous approaches. The mean squared error (MSE) obtained using Adaboost is higher compared to the MSE obtained using other approaches.

* Q: Explain the impact of the loss and the learning_rate hyperparameters on the accuracy. Which hyperparameter setting gives you the best value? Is this the same as the default settings in sklearn?

* A: The loss hyperparameter in Adaboost determines the loss function to be used when updating the weights of the samples. The choices are "linear", "square", and "exponential". The learning_rate hyperparameter controls the contribution of each weak learner to the final combination. Lower learning rates generally result in more accurate models, but they may require more weak learners (boosting rounds) to achieve the same performance.

* Q: Explain the impact of the hyperparameters on the training time. 
* A: The training time in Adaboost is influenced by various factors, including the number of weak learners (boosting rounds), the complexity of the weak learners, and the size of the dataset. Generally, increasing the number of boosting rounds and the complexity of the weak learners tends to increase training time because it requires fitting more weak learners. However, Adaboost training is sequential, meaning each weak learner is trained sequentially, which can lead to longer training times compared to parallelizable algorithms like Random Forest.